In [ ]:
import pandas as pd
import numpy as np

df_original = pd.read_csv("../train_data.csv")
n_original = df_original.shape[0]
df_submit = pd.read_csv("../sample_submission.csv")
df = pd.concat([df_original, df_submit], axis=0).reset_index(drop=True)

# Features from GRU
df = pd.read_csv("../GRU_features_predict_only.csv").merge(
    df,
    on='id'
)


In [ ]:
from multimolecule import RnaTokenizer, RiNALMoModel

tokenizer = RnaTokenizer.from_pretrained('multimolecule/rinalmo')
model = RiNALMoModel.from_pretrained('multimolecule/rinalmo')


In [ ]:
extracted_features = []

for i in range(df.shape[0]):
    if i%10==0:
        print(i)
    input = tokenizer(RNA_seq[i], return_tensors='pt')
    output = model(**input)
    extracted_features.append(output.pooler_output.squeeze().detach().numpy())


In [ ]:
retval = pd.DataFrame( np.array(extracted_features), columns=['RiNALMo_feature_'+str(i) for i in range(1,1281)] )
retval['id'] = df['id']
retval.to_csv('../RiNalMo_features.csv')


In [ ]:
retval.head()

In [ ]:
# Dimension Reduction
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

df = pd.read_csv("../RiNALMo_features.csv", index_col=0)

pca = PCA(n_components=50)
X = np.array( df[[x for x in df.columns if 'RiNALMo_feature' in x]] )
pca.fit(X)

X_pca = X @ pca.components_.T


In [ ]:
retval = pd.DataFrame( X_pca, columns=['RiNALMo_feature_PCA_'+str(i) for i in range(1,1+X_pca.shape[1])] )
retval['id'] = df['id']
retval.to_csv('../RiNalMo_features_PCA.csv')

In [ ]:
import torch
from multimolecule import RnaTokenizer, RnaFmForNucleotidePrediction

tokenizer = RnaTokenizer.from_pretrained('multimolecule/mrnafm')
model = RnaFmForNucleotidePrediction.from_pretrained('multimolecule/mrnafm')


In [ ]:
RNA_seq = list(df['siRNA_antisense_seq'])
output_list = []
max_len = 24

with torch.no_grad():
    for i in range(len(RNA_seq)):
        if i%100==0:
            print(i)
        x = RNA_seq[i]
        x = x + ''.join(['-']*(max_len-len(x)))
        input = tokenizer(x, return_tensors='pt')
        output = model(**input)
        output_list.append( list(output.logits.squeeze().numpy()) )


In [ ]:
retval = pd.DataFrame( np.array(output_list), columns=['mRNAFM_feature_'+str(i) for i in range(1,9)] )
retval['id'] = df['id']
retval.to_csv('../mrnafm_features.csv')


In [ ]:
retval

In [ ]:
min_len = min( len(x) for x in RNA_seq )
min_len